<a href="https://colab.research.google.com/github/rudraxx/pytorch/blob/main/Vision_Transformer_in_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vision Transformer (ViT)

I will be implementing the Vision Transformer network from scratch. We will train the network on CIFAR-10 data.

Link to the original Paper:


1.   [Attention is All you need](https://arxiv.org/pdf/1706.03762)
2.   [An Image is Worth 16x16 Words: Transformers for Image Recognition at Scale
](https://arxiv.org/abs/2010.11929)



Articles used as reference:


1.   [Article on Medium](https://towardsdatascience.com/implementing-vision-transformer-vit-from-scratch-3e192c6155f0)
2.   List item



Overview of the VIT Architecture
![image.png](https://miro.medium.com/v2/resize:fit:1400/format:webp/1*Q-mBZkDz7TUnVGw1KPwqOA.png)

In [3]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

import torch.optim as optim


import math


In [ ]:
#Set up some of the hyper parameters

#Load the config
config ={"image_size": 128, "patch_size": 16,
         "num_channels":3, "hidden_size": 16,
         "hidden_dropout_prob": 0.2,
         "num_attention_heads":4,
         "qkv_bias": False,
         "attention_probs_dropout_prob": 0.2,
         "hidden_dropout_prob": 0.2,
         "intermediate_size": 8,
         "num_hidden_layers": 2}


In [ ]:
# Set the device type

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Selected device: {device}")


Selected device: cpu


**Stages of the model pipeline**
1. Convert images into patches
2. Run the patches through linear layer to get patch embeddings. Layer weights are learnt.
3. Add the CLS token as the first token for all instances in the batch
3.  Get positional embeddings( sin/cos transform)
4. Input to the transformer is the sum of patch and positional embeddings



In [ ]:
#Convert images into patches
class PatchEmbeddings(nn.Module):
  def __init__(self, config):
    super().__init__()

    self.image_size   = config["image_size"] # Size of the incoming images ih xiw
    self.patch_size   = config["patch_size"] # What is the size of the patch hxw
    self.num_channels = config["num_channels"] # ch
    self.hidden_size  = config["hidden_size"] # d_hidden

    #Calculate the number of patches from the image and patch size
    self.num_patches = (self.image_size // self.patch_size)**2 # p x p

    #Create the projection to convert the images into patches
    # This layer should take each patch and convert it into a 1D vector of size (1, d_hidden)
    self.projection = nn.Conv2d(self.num_channels, self.hidden_size,
                                kernel_size=self.patch_size,
                                stride=self.patch_size)


  def forward(self, x):

    # x is of shape: (B X num_channels X image_size X image_size)
    # Required output is of shape: (B X num_patches X d_hidden)

    #1) (B X num_channels X image_size X image_size) -> (B X d_hidden X self.num_patches X self.num_patches)
    x = self.projection(x)
    #2) (B X d_hidden X self.num_patches X self.num_patches) -> (B X d_hidden X self.num_patches*self.num_patches)
    x = x.flatten(2)
    #3) (B X d_hidden X self.num_patches*self.num_patches) -> (B X self.num_patches*self.num_patches X d_hidden)
    x=x.transpose(1,2)

    return x



In [ ]:
#Test out the patch embeddings class
x = torch.randn(32,config["num_channels"], config["image_size"], config["image_size"])
print(f"Input tensor shape to the patch embedding model: \n {x.shape}. (B X num_channels X image_size X image_size)")


patch_embedding = PatchEmbeddings(config)

x = patch_embedding(x)
print(f"After the patch embedding:\n {x.shape}. (B X num_patches X d_hidden)")



Input tensor shape to the patch embedding model: 
 torch.Size([32, 3, 128, 128]). (B X num_channels X image_size X image_size)
After the patch embedding:
 torch.Size([32, 64, 17]). (B X num_patches X d_hidden)


In [ ]:
# Add the CLS token to the beginning of each sequence
class Embeddings(nn.Module):
  def __init__(self, config):
    super().__init__()

    self.patch_embeddings = PatchEmbeddings(config)

    #Create a learnable [CLS] token. This is added before the first patch,
    # so should be the same dimension of the other patches
    self.cls_token = nn.Parameter(torch.randn(1, config["hidden_size"]))

    # Position embedding is added(summed up) with the B X CLS+num_patches X d_hidden,
    # so ensure the shape of position_embedding takes this into account
    self.position_embeddings = \
      nn.Parameter(torch.randn(1,self.patch_embeddings.num_patches + 1, config["hidden_size"]))

    # Drop out layer
    self.dropout = nn.Dropout(config["hidden_dropout_prob"])

  def forward(self, x):

    batch_size = x.shape[0]
    #Get the patch embeddings
    x = self.patch_embeddings(x)

    #Add the CLS token to every batch item
    cls_tokens = self.cls_token.unsqueeze(0).repeat(batch_size, 1, 1)  # Shape: (B, 1, d_hidden)
    # Concat the cls token. Note that the size of the hidden_size stays the same. The number of patches/tokens increases by 1.
    # because CLS is added as the first token.
    x = torch.cat((cls_tokens, x), dim=1)

    #Add positional embedding
    x = x + self.position_embeddings

    # Dropout
    x = self.dropout(x)

    return x


In [ ]:
#Test code to try the math for addition of the cls to the batch
cls_x1 = torch.randn(1,10)
print("cls_x1", cls_x1.shape)

expanded_cls = cls_x1.unsqueeze(0).repeat(32,1,1)
print("expanded_cls", expanded_cls.shape)

x = torch.randn(32,200, 10)
print("x", x.shape)

cat_result = torch.cat((expanded_cls, x), dim=1)
print("cat_result", cat_result.shape)



cls_x1 torch.Size([1, 10])
expanded_cls torch.Size([32, 1, 10])
x torch.Size([32, 200, 10])
cat_result torch.Size([32, 201, 10])


In [ ]:
#Test out the embeddings
embeddings = Embeddings(config)
print(embeddings)


Embeddings(
  (patch_embeddings): PatchEmbeddings(
    (projection): Conv2d(3, 17, kernel_size=(16, 16), stride=(16, 16))
  )
  (dropout): Dropout(p=0.2, inplace=False)
)


The CLS + Position + Patch embeddings have been created. Next step is to create the Transformer Encoder model

# Multi-head Attention Module

The Attention module takes sequence of embeddings as input and computes query, key and value vectors for each embedding

In [ ]:
class AttentionHead(nn.Module):
  """
  A single attention head
  Used in MultiAttentionHead module

  """

  def __init__(self, hidden_size, attention_head_size, prob_dropout, bias=True):
    super().__init__()

    self.hidden_size = hidden_size
    self.attention_head_size = attention_head_size

    #Create the query, key and value projection layers
    self.query  = nn.Linear(hidden_size, attention_head_size, bias=bias)
    self.key    = nn.Linear(hidden_size, attention_head_size, bias=bias)
    self.value  = nn.Linear(hidden_size, attention_head_size, bias=bias)

    self.dropout = nn.Dropout(prob_dropout)

  def forward(self, x):
    # Shape of x: (B x 1+num_patches x hidden_size)
    #Project the input into query, key, value
    # Since the same x vector is used to generate the Q,K,V values,
    # this is called self-attention
    query = self.query(x) # (B x 1+num_patches x attention_head_size)
    key   = self.key(x)   # (B x 1+num_patches x attention_head_size)
    value = self.value(x) # (B x 1+num_patches x attention_head_size)

    #Calculate the attention scores
    # softmax(Q*K.T/sqrt(head_size)) *V
    # print("query.shape", query.shape)
    # print("key.shape", key.shape)
    # print("value.shape", value.shape)

    attention_scores = query @ key.transpose(-1,-2) # Q*K.T
    attention_scores = attention_scores/self.attention_head_size**0.5 # Q*K.T/sqrt(d_head)
    # print("attention_score for 0th item, 0th token pre softmax:", attention_scores[0,0,:])

    #For every token( dim=1), get the softmax across all the other tokens.
    # At this stage the dims are B X 1+num_patches X 1+num_patches.
    # Take the softmax across the last dim, because each entry in dim=1
    # represents the score wrt all the other tokens
    attention_probs = nn.functional.softmax(attention_scores, dim=-1)
    # print("attention_score for 0th item, 0th token post softmax:", attention_scores[0,0,:])
    # print("sum of attention_score for 0th item, 0th token post softmax:", torch.sum(attention_scores[0,0,:]))

    # Dropout
    attention_probs = self.dropout(attention_probs)

    #Now the scores are of the size: B X 1+num_patches X 1+num_patches,
    # and they represent the relative weight for other tokens. Multiply this
    # to the value matrix to get the update
    attention_output = attention_probs@value
    # print("attention_scores.shape", attention_scores.shape)

    return (attention_output, attention_probs)


# Test
hidden_size= config["hidden_size"]#14
print(hidden_size)

attention_head_size = config["hidden_size"] // config["num_attention_heads"]
# attention = AttentionHead(, ,
#                           prob_dropout=0.2)

attention = AttentionHead(hidden_size, attention_head_size,
                          prob_dropout=0.2)

x = torch.randn(32, 65, hidden_size)
print(f"hidden_size={hidden_size}")
print(f"Input shape: {x.shape}")
print(f"attention_head_size={attention_head_size}")
(attention_output, attention_probs) = attention(x)

print(f"Output attention_output.shape = {attention_output.shape}, attention_probs: {attention_probs.shape}")

16
hidden_size=16
Input shape: torch.Size([32, 65, 16])
attention_head_size=4
Output attention_output.shape = torch.Size([32, 65, 4]), attention_probs: torch.Size([32, 65, 65])


In [ ]:
config["hidden_size"]

16

In [ ]:
class MultiHeadAttention(nn.Module):
  """
  Multi-haed attention module.
  Used in Transformer Encoder module
  """
  def __init__(self, config):
    super().__init__()
    self.hidden_size = config["hidden_size"]
    self.num_attention_heads = config["num_attention_heads"]
    # The attention head size is the hidden size divided by the number of attention heads
    self.attention_head_size = self.hidden_size//self.num_attention_heads

    #Recalculate the total size of head. If the hidden size is not a integer
    # multiple of num_attention_heads, the all_head_size value can be different
    # than self.hidden_size
    self.all_head_size = self.attention_head_size * self.num_attention_heads

    # Decide if bias is used in the Q,K,V projection layers
    self.qkv_bias = config["qkv_bias"]

    #Create the list of attention heads
    self.heads = nn.ModuleList([])

    for _ in range(self.num_attention_heads):
      head = AttentionHead(self.hidden_size, self.attention_head_size,
                           config["attention_probs_dropout_prob"], self.qkv_bias)
      self.heads.append(head)

    #Create a linear layer to project the attention output back to the hidden size
    self.output_projection = nn.Linear(self.all_head_size, self.hidden_size)
    self.output_dropout = nn.Dropout(config["hidden_dropout_prob"])

  def forward(self, x, output_attentions=False):
    print(x.shape)
    # Calculate the attention output for each attention head
    attention_result = [head(x) for head in self.heads]

    #Concat the results along the last dim. Recombine to get back the "hidden_size" in last dim
    attention_output = torch.cat([attention_outputs for (attention_outputs, _) in attention_result], dim=-1)

    #Project the concatenated attention output back to hidden_size
    attention_output = self.output_projection(attention_output)
    attention_output = self.output_dropout(attention_output)

    #Return the attention output and probabilities if needed
    if not output_attentions:
      return (attention_output, None)
    else:
      attention_probs = torch.cat([attention_prob for (_, attention_prob) in attention_result], dim=1)
      return (attention_output, attention_probs)



# Test
mha = MultiHeadAttention(config)

# # Test
# attention = AttentionHead(config["hidden_size"], config["attention_head_size"],
#                           prob_dropout=0.2)

x = torch.randn(32, 65, config["hidden_size"])
print(f"hidden_size={hidden_size}")
print(f"Input shape: {x.shape}")
print(f"attention_head_size={attention_head_size}")
attention_output, attention_probs = mha(x, output_attentions=True)

print(f"Output attention_output.shape = {attention_output.shape}, attention_probs: {attention_probs.shape}")

hidden_size=16
Input shape: torch.Size([32, 65, 16])
attention_head_size=4
torch.Size([32, 65, 16])
Output attention_output.shape = torch.Size([32, 65, 16]), attention_probs: torch.Size([32, 260, 65])


# Transformer Encoder

![Encoder structure](https://miro.medium.com/v2/resize:fit:1400/format:webp/0*iHJ8nTDR1CpOSb-o.png)

In [ ]:
class NewGELUActivation(nn.Module):
    """
    Implementation of the GELU activation function currently in Google BERT repo (identical to OpenAI GPT). Also see
    the Gaussian Error Linear Units paper: https://arxiv.org/abs/1606.08415
    """

    def forward(self, input):
        return (
            0.5
            * input
            * (
                1.0
                + torch.tanh(
                    math.sqrt(2.0 / math.pi)
                    * (input + 0.044715 * torch.pow(input, 3.0))
                )
            )
        )


class MLP(nn.Module):
  def __init__(self, config):
    super().__init__()
    self.dense_1 = nn.Linear(config["hidden_size"], config['intermediate_size'])
    self.activation = NewGELUActivation()
    self.dense_2 = nn.Linear(config['intermediate_size'], config["hidden_size"])
    self.dropout = nn.Dropout(config["hidden_dropout_prob"])

  def forward(self, x):
    x = self.dense_1(x)
    x = self.activation(x)
    x = self.dense_2(x)
    x = self.dropout(x)

    return x




In [ ]:
class Block(nn.Module):
  '''
  A single transformer block
  '''

  def __init__(self, config):
    super().__init__()
    self.attention = MultiHeadAttention(config)

    self.layernorm_1 = nn.LayerNorm(config["hidden_size"])
    self.mlp = MLP(config)
    self.layernorm_2 = nn.LayerNorm(config["hidden_size"])

  def forward(self, x, output_attentions=False):
    xres_1 = x # Keep track for skip connection
    x = self.layernorm_1(xres_1)
    # Self attention
    (attention_output, attention_probs) = self.attention(x, output_attentions=output_attentions)

    #Add the first skip connection
    xres_2 = attention_output + xres_1 # Add the skip connection
    x = self.layernorm_1(xres_2)

    #FF Network
    x = self.mlp(x)

    #Add the second skip connection
    x = x + xres_2

    # Return the output and attention probs if asked for:

    if output_attentions:
      return (x, attention_probs)
    else:
      return (x, None)



In [ ]:
# The transformer encoder stacks multiple Blocks on top of each other

class Encoder(nn.Module):
  def __init__(self, config):
    super().__init__()

    self.blocks = nn.ModuleList([])
    for _ in range(config["num_hidden_layers"]):
      self.blocks.append(Block(config))

  def forward(self, x, output_attentions=False):

    #Calculate the transformer block's output for each block
    all_attentions = []
    for block in self.blocks:
      x, attention_probs = block(x, output_attentions)
      if output_attentions:
        all_attentions.append(attention_probs)
    # Return the encoder's output and the attention probabilities

    if output_attentions:
      return (x, all_attentions)
    else:
      return (x, None)




# ViT for image classification

In [ ]:
class ViTClassification(nn.Module):
  def __init__(self, config):
    super().__init__()

    self.config = config
    self.image_size = config["image_size"]
    self.hidden_size = config["hidden_size"]
    self.num_classes = config["num_classes"]


    #Create the embedding model
    self.embeddings = Embeddings(config)

    #Encoder
    self.encoder = Encoder(config)

    #Linear layer to project the encoder's output to number of classes
    self.mlp_final = nn.Linear(self.hidden_size, self.num_classes) #(B x 1+num_patches x hidden_size)

    #Initialize the weights
    # self.apply(self._init_weights)

  def forward(self, x, output_attentions=False):

    # Get the embeddings for the imput
    x = self.embeddings(x)

    #Encoded values
    (encoder_output, all_attentions) = self.encoder(x, output_attentions)

    # Calculate the logits, take the [CLS] tokens output as the feature vector for the final classification MLP
    logits = self.mlp_final(encoder_output[:,0])

    #Return the logits and attention probs
    if output_attentions:
      return (logits, all_attentions)
    else:
      return (logits, None)




In [ ]:
#Test the full model
x = torch.randn(12, 3, 32,32)
config = {
    "patch_size": 4,  # Input image size: 32x32 -> 8x8 patches
    "hidden_size": 48,
    "num_hidden_layers": 4,
    "num_attention_heads": 4,
    "intermediate_size": 4 * 48, # 4 * hidden_size
    "hidden_dropout_prob": 0.0,
    "attention_probs_dropout_prob": 0.0,
    "initializer_range": 0.02,
    "image_size": 32,
    "num_classes": 10, # num_classes of CIFAR10
    "num_channels": 3,
    "qkv_bias": True,
    "use_faster_attention": True,
}

vit = ViTClassification(config)

logits, attention_probs = vit(x, output_attentions=False)

print(logits.shape)


torch.Size([12, 65, 48])
torch.Size([12, 65, 48])
torch.Size([12, 65, 48])
torch.Size([12, 65, 48])
torch.Size([12, 10])


# CIFAR10 dataset

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
    ])

ds_train = datasets.CIFAR10(root="./data", train=True,
                            download=True, transform=transform)


In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')